# Estimação de Densidade por Kernel e Detecção de Outliers em Preços do Airbnb em Lisboa

## Objetivo
Este notebook estima a distribuição dos preços de acomodações do Airbnb em Lisboa usando **Kernel Density Estimation (KDE)** com kernel Gaussiano. A largura de banda é escolhida por **máxima verossimilhança leave-one-out (MLKDE)** e a densidade resultante é utilizada para identificar preços improváveis (outliers).

## 1. Título e objetivo
Este estudo busca:
- Estimar a densidade dos preços usando KDE com kernel Gaussiano.
- Escolher a largura de banda (bandwidth) que maximiza a log-verossimilhança leave-one-out (MLKDE).
- Usar a densidade estimada para destacar preços com baixa probabilidade (outliers).

## 2. Importação de bibliotecas
Carregamos as bibliotecas necessárias para manipulação de dados, cálculo numérico e visualização.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import gzip
import io
import re
import shutil
import urllib.request

plt.style.use("seaborn-v0_8-whitegrid")
%matplotlib inline


## 3. Carregamento e preparação dos dados
O conjunto de dados é o `listings.csv` do **Inside Airbnb** (Lisboa). O notebook baixa automaticamente a versão mais recente disponível no site e salva como `data/listings_lisboa.csv` se o arquivo ainda não existir localmente. A análise usa apenas a coluna de preços (`price`).

Passos de limpeza:
1. Remover linhas sem preço.
2. Converter strings de preço (por exemplo, `"$120.00"`) para valores numéricos.
3. Filtrar preços não positivos.
4. Criar `log_price = log(preço)`, o que reduz a influência de caudas longas e facilita a modelagem.


### Download automático do dataset
A função abaixo consulta `http://insideairbnb.com/get-the-data/` para encontrar o link mais recente de Lisboa, baixa o arquivo `listings.csv.gz`, descompacta e salva em `data/listings_lisboa.csv`. Se o arquivo já existir, o download é pulado.


In [ ]:
def download_latest_lisbon_listing(destination: Path) -> Path:
    destination.parent.mkdir(parents=True, exist_ok=True)
    if destination.exists():
        print(f"Arquivo já encontrado em {destination}. Usando versão local.")
        return destination

    index_url = "http://insideairbnb.com/get-the-data/"
    with urllib.request.urlopen(index_url, timeout=15) as resp:
        html = resp.read().decode("utf-8", errors="ignore")

    match = re.search(
        r"https://data\.insideairbnb\.com/portugal/lisbon/lisbon/[0-9-]+/data/listings\.csv\.gz",
        html,
    )
    if not match:
        raise RuntimeError("Não foi possível localizar o link de Lisboa no Inside Airbnb.")

    download_url = match.group(0)
    request = urllib.request.Request(download_url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(request, timeout=60) as resp:
        compressed_bytes = io.BytesIO(resp.read())

    with gzip.open(compressed_bytes, "rt", encoding="utf-8") as gz, destination.open("w", encoding="utf-8") as out:
        shutil.copyfileobj(gz, out)

    print(f"Arquivo baixado de {download_url}")
    print(f"Salvo como {destination}")
    return destination


In [ ]:
data_path = Path("data/listings_lisboa.csv")
try:
    download_latest_lisbon_listing(data_path)
except Exception as exc:
    raise RuntimeError(
        "Não foi possível baixar o dataset automaticamente. Baixe o listings.csv de Lisboa no Inside Airbnb e salve como data/listings_lisboa.csv."
    ) from exc

# Leitura e cópia do conjunto de dados original
raw_df = pd.read_csv(data_path)
if "price" not in raw_df.columns:
    raise KeyError("A coluna 'price' não foi encontrada no arquivo fornecido.")

# Limpeza da coluna de preço
prices = (
    raw_df["price"]
    .astype(str)
    .str.replace(r"[^0-9,\.]", "", regex=True)  # remove símbolos monetários e outros caracteres
    .str.replace(",", "", regex=False)  # remove separadores de milhar
)
prices = pd.to_numeric(prices, errors="coerce")

clean_df = raw_df.copy()
clean_df["price"] = prices
clean_df = clean_df.dropna(subset=["price"]).copy()
clean_df = clean_df[clean_df["price"] > 0].copy()
clean_df["log_price"] = np.log(clean_df["price"])
clean_df = clean_df.reset_index(drop=True)

print(f"Total de entradas após limpeza: {len(clean_df)}")
clean_df[["price", "log_price"]].head()


## 4. Análise exploratória simples
A seguir mostramos estatísticas descritivas e histogramas para `price` e `log_price`, destacando assimetria e caudas longas.

In [ ]:
estatisticas = clean_df[["price", "log_price"]].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
estatisticas


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].hist(clean_df["price"], bins=40, color="#4c72b0", edgecolor="black")
axes[0].set_title("Histograma de price")
axes[0].set_xlabel("Preço")
axes[0].set_ylabel("Frequência")

axes[1].hist(clean_df["log_price"], bins=40, color="#55a868", edgecolor="black")
axes[1].set_title("Histograma de log_price")
axes[1].set_xlabel("log(preço)")
axes[1].set_ylabel("Frequência")

plt.tight_layout()
plt.show()


## 5. Implementação do KDE e do MLKDE

**Kernel Density Estimation (KDE)** aproxima a função densidade de probabilidade dos dados. Para cada ponto de avaliação \(x\), a estimativa é dada por:

\[\hat{f}(x) = rac{1}{n h} \sum_{i=1}^{n} K\left( rac{x - x_i}{h} ight)\]

- \(K\) é o kernel Gaussiano (simétrico, suave e com integral 1).
- \(h\) (bandwidth) controla a suavização: valores pequenos produzem curvas muito onduladas; valores grandes podem esconder detalhes.

Para escolher \(h\), usamos **máxima verossimilhança leave-one-out (MLKDE)**:
- Para cada \(x_i\), calculamos \(\hat{f}_{-i}(x_i)\) excluindo o próprio ponto.
- Somamos \(\log(\hat{f}_{-i}(x_i))\) para obter a log-verossimilhança.
- O \(h\) ótimo maximiza essa soma.

In [ ]:
def gaussian_kernel(u: np.ndarray) -> np.ndarray:
    """Kernel Gaussiano padrão."""
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * u ** 2)


def kde_density(x_grid: np.ndarray, data: np.ndarray, bandwidth: float) -> np.ndarray:
    """KDE univariado com kernel Gaussiano."""
    n = data.shape[0]
    scaled_diff = (x_grid[:, None] - data[None, :]) / bandwidth
    kernel_vals = gaussian_kernel(scaled_diff)
    density = kernel_vals.mean(axis=1) / bandwidth
    return density


def loo_log_likelihood(data: np.ndarray, bandwidth: float) -> float:
    """Log-verossimilhança leave-one-out para um bandwidth dado."""
    n = data.shape[0]
    if bandwidth <= 0:
        return -np.inf
    scaled_diff = (data[:, None] - data[None, :]) / bandwidth
    kernel_vals = gaussian_kernel(scaled_diff)
    np.fill_diagonal(kernel_vals, 0.0)
    density_i = kernel_vals.sum(axis=1) / ((n - 1) * bandwidth)
    density_i = np.maximum(density_i, 1e-12)  # evita log(0)
    return float(np.sum(np.log(density_i)))


log_prices = clean_df["log_price"].to_numpy()
std_log = np.std(log_prices, ddof=1) if len(log_prices) > 1 else 1.0

# Intervalo de busca para h
h_min = max(0.05, 0.1 * std_log)
h_max = max(h_min * 2, 1.5 * std_log)
h_values = np.linspace(h_min, h_max, 40)

log_liks = [loo_log_likelihood(log_prices, h) for h in h_values]
best_idx = int(np.argmax(log_liks))
best_h = h_values[best_idx]

print(f"Bandwidth ótimo (MLKDE): {best_h:.4f}")


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(h_values, log_liks, marker="o", color="#4c72b0")
ax.axvline(best_h, color="#c44e52", linestyle="--", label=f"h ótimo = {best_h:.4f}")
ax.set_xlabel("Bandwidth (h)")
ax.set_ylabel("Log-verossimilhança LOO")
ax.set_title("Busca do bandwidth por MLKDE")
ax.legend()
plt.show()


O valor de \(h\) que maximiza a log-verossimilhança oferece o melhor equilíbrio entre suavização e fidelidade aos dados segundo o critério LOO.

## 6. KDE final com o bandwidth ótimo
Usamos o \(h\) selecionado para estimar a densidade em um grid ao redor de `log_price` e comparamos a curva KDE com o histograma (densidade empírica).

In [ ]:
margin = 0.5 * std_log if std_log > 0 else 0.5
x_grid = np.linspace(log_prices.min() - margin, log_prices.max() + margin, 200)
density_grid = kde_density(x_grid, log_prices, best_h)

fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(clean_df["log_price"], bins=40, density=True, color="#c7e9c0", edgecolor="#4c72b0", alpha=0.8, label="Histograma (densidade)")
ax.plot(x_grid, density_grid, color="#1b9e77", linewidth=2.5, label="KDE com h ótimo")
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade")
ax.set_title("Densidade estimada vs. histograma")
ax.legend()
plt.show()


A curva KDE suavizada acompanha a distribuição observada de `log_price`, revelando a forma geral da densidade e destacando possíveis regiões com menor probabilidade.

## 7. Detecção de outliers com base na densidade
Pontos com densidade estimada muito baixa indicam preços improváveis. Vamos usar um critério configurável: marcar como outlier qualquer observação cuja densidade estimada esteja abaixo de um percentil definido (por padrão, 1%). Isso deixa claro o limiar usado e permite ajustar a sensibilidade da detecção.


In [ ]:
percentil_outlier = 1.0  # percentil usado para definir o limiar de densidade

densidades_nos_pontos = kde_density(log_prices, log_prices, best_h)
limiar = np.percentile(densidades_nos_pontos, percentil_outlier)
clean_df["kde_density"] = densidades_nos_pontos
clean_df["outlier"] = clean_df["kde_density"] < limiar

n_outliers = int(clean_df["outlier"].sum())
proporcao_outliers = n_outliers / len(clean_df) if len(clean_df) > 0 else 0.0

resumo_outliers = clean_df[["price", "log_price", "kde_density", "outlier"]].copy()
resumo_outliers_ordenado = resumo_outliers.sort_values("kde_density", ascending=True)

print(f"Limiar de densidade (percentil {percentil_outlier}%): {limiar:.6f}")
print(f"Total de outliers: {n_outliers}")
print(f"Proporção de outliers: {proporcao_outliers:.2%}")


Chamamos de **outliers** as acomodações que caem na região de densidade mais baixa da curva — por padrão, os **1% de preços menos prováveis segundo o modelo**. Esse critério usa o percentil `percentil_outlier` para definir o limiar de densidade.

Estas são as observações mais improváveis segundo a densidade estimada (ordenadas da menor para a maior densidade). A tabela ajuda a enxergar os preços que ficam na cauda da distribuição — os 'preços mais estranhos' para o modelo.

In [ ]:
top_outliers = resumo_outliers_ordenado.head(20)
display(top_outliers)


### Visualização dos outliers
Os gráficos a seguir destacam onde os outliers aparecem na escala de preços original e na relação entre o preço e a densidade estimada.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
contagens, bins, _ = ax.hist(
    clean_df["price"], bins=50, density=True, color="#d0e1f9", edgecolor="#4c72b0", alpha=0.85, label="Distribuição de preços"
)

p5, p95 = np.percentile(clean_df["price"], [5, 95]) if len(clean_df) > 0 else (0, 0)
ax.axvspan(p5, p95, color="#b2df8a", alpha=0.3, label="Faixa típica (5% a 95%)")

outlier_prices = clean_df.loc[clean_df["outlier"], "price"]
if not outlier_prices.empty:
    y_max = contagens.max() if len(contagens) else 0.0
    y_offset = -0.02 * y_max if y_max > 0 else -0.001
    ax.scatter(
        outlier_prices,
        np.full_like(outlier_prices, y_offset),
        color="#e84a5f",
        alpha=0.9,
        marker="v",
        s=50,
        label="Outliers (densidade < limiar)"
    )
    ax.set_ylim(y_offset * 1.2, ax.get_ylim()[1] * 1.05)

ax.set_xlabel("Preço (euros)")
ax.set_ylabel("Densidade")
ax.set_title("Distribuição dos preços com outliers destacados")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
cores = np.where(clean_df["outlier"], "#e84a5f", "#4c72b0")

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(
    clean_df["price"],
    clean_df["kde_density"],
    c=cores,
    alpha=0.8,
    edgecolor="white",
    linewidth=0.6,
    label="Observações"
)
ax.axhline(limiar, color="#e84a5f", linestyle="--", linewidth=1.5, label=f"Limiar de densidade ({percentil_outlier}% mais baixo)")
ax.set_xscale("log")
ax.set_xlabel("Preço (escala log)")
ax.set_ylabel("Densidade estimada")
ax.set_title("Preços vs. densidade — pontos abaixo da linha são outliers")
ax.legend()
plt.tight_layout()
plt.show()


### Por que esses pontos são outliers?
- A maior parte das acomodações concentra-se em uma faixa típica de preços.
- Os pontos marcados como outliers ficam na ponta da cauda da distribuição: preços muito baixos ou muito altos em relação à maioria.
- Eles podem refletir acomodações de luxo, anúncios com erro de preço ou estratégias agressivas de precificação — situações raras e pouco prováveis.

## 8. Comparação do bandwidth por máxima verossimilhança com regras clássicas
Vamos comparar o bandwidth ótimo obtido pelo MLKDE com duas regras clássicas para KDE univariado: Silverman e Scott. Além de listar cada \(h\), avaliamos a log-verossimilhança leave-one-out para ver qual escolha se ajusta melhor aos dados.

In [ ]:
n = len(log_prices)
iqr_log = np.subtract(*np.percentile(log_prices, [75, 25])) if n > 0 else 0.0
sigma_log = np.std(log_prices, ddof=1) if n > 1 else 0.0

h_silverman = 0.9 * min(sigma_log, iqr_log / 1.34) * n ** (-1 / 5) if n > 1 else np.nan
h_scott = sigma_log * n ** (-1 / 5) if n > 1 else np.nan

h_candidates = {"MLKDE": best_h, "Silverman": h_silverman, "Scott": h_scott}


def loglik_loo(h):
    return loo_log_likelihood(log_prices, h) if np.isfinite(h) else -np.inf

comparacao_h = pd.DataFrame({
    "metodo": list(h_candidates.keys()),
    "h": list(h_candidates.values()),
    "log_verossimilhanca_loo": [loglik_loo(h) for h in h_candidates.values()]
}).sort_values("log_verossimilhanca_loo", ascending=False)

comparacao_h


A tabela resume as larguras de banda testadas e a log-verossimilhança leave-one-out de cada uma (quanto maior, melhor). O gráfico compara as curvas KDE resultantes.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(clean_df["log_price"], bins=40, density=True, color="#e0f3db", edgecolor="#4c72b0", alpha=0.7, label="Histograma de log(preço)")

cores_metodos = {"MLKDE": "#1b9e77", "Silverman": "#d95f02", "Scott": "#7570b3"}
for metodo, h in h_candidates.items():
    if np.isfinite(h):
        dens = kde_density(x_grid, log_prices, h)
        ax.plot(x_grid, dens, color=cores_metodos.get(metodo, "black"), linewidth=2, label=f"{metodo} (h = {h:.4f})")
ax.set_xlabel("log(preço)")
ax.set_ylabel("Densidade")
ax.set_title("Curvas KDE para diferentes larguras de banda")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
outlier_sets = {}
for metodo, h in h_candidates.items():
    if np.isfinite(h):
        densidades = kde_density(log_prices, log_prices, h)
        limite = np.percentile(densidades, percentil_outlier)
        indices_outliers = set(clean_df.index[densidades < limite])
        outlier_sets[metodo] = indices_outliers

resumo_outliers_metodos = pd.DataFrame([
    {"metodo": metodo, "qtd_outliers": len(indices), "outliers_com_MLKDE": len(indices & outlier_sets.get("MLKDE", set()))}
    for metodo, indices in outlier_sets.items()
]) if outlier_sets else pd.DataFrame()

outliers_comuns_todos = len(set.intersection(*outlier_sets.values())) if len(outlier_sets) > 1 else (len(next(iter(outlier_sets.values()))) if outlier_sets else 0)

display(resumo_outliers_metodos)
print(f"Outliers comuns a todos os métodos: {outliers_comuns_todos}")


Os valores de log-verossimilhança indicam qual largura de banda se ajusta melhor aos dados: o método no topo da tabela maximiza essa métrica e tende a reproduzir melhor a forma empírica da distribuição. As curvas geralmente ficam parecidas, mas pequenas diferenças de suavização aparecem nas caudas. Bandwidths menores destacam mais picos e podem gerar mais outliers; bandwidths maiores suavizam a curva e escondem alguns extremos.

### Quem performou melhor?
- O método listado em primeiro lugar na tabela tem a maior log-verossimilhança leave-one-out, ou seja, ele explica melhor os dados segundo esse critério.
- Visualmente, as curvas diferem pouco no centro, mas Silverman/Scott podem suavizar mais as caudas, enquanto o MLKDE tende a seguir de perto os picos observados.
- Dizer que o MLKDE é "melhor" significa que ele maximiza a verossimilhança; em contextos exploratórios, regras clássicas ainda podem ser suficientes quando simplicidade e rapidez são prioridades.

## 9. Conclusão
- Estimamos a densidade dos preços do Airbnb em Lisboa via KDE com kernel Gaussiano.
- O bandwidth escolhido por máxima verossimilhança leave-one-out (MLKDE) superou as regras clássicas na log-verossimilhança e guiou a marcação de outliers.
- A densidade estimada identificou outliers como os "1% mais improváveis" da distribuição: preços muito fora do padrão da maioria dos anúncios.
- Um bandwidth maior tenderia a suavizar a curva e reduzir a quantidade de outliers sinalizados; um bandwidth menor realçaria picos e poderia aumentar a detecção de pontos extremos.
- A transformação logarítmica estabiliza a variabilidade e facilita a visualização de preços incomuns; análises futuras podem segmentar por bairro, tipo de acomodação ou adicionar variáveis para critérios multivariados.